In [1]:
import pandas as pd
df = pd.read_csv("ai-light-bulb-dataset.csv")
df

,Light Sensor,Time,Light State
0,0.0,00:00,Off
1,0.0,01:00,Off
2,0.0,02:00,Off
3,0.0,03:00,Off
4,0.0,04:00,Off
5,0.1,04:00,Off
6,0.1,05:00,On
7,0.2,06:00,On
8,0.3,07:00,On
9,0.4,08:00,Off


Before we build the Naive Bayes model, I want to group light sensor values into four levels as follows:

if 0.00≤ Light Sensor ≤0.25 ---> 0::
if 0.26≤ Light Sensor ≤0.50 ---> 1::
if 0.51≤ Light Sensor ≤0.75 ---> 2::
if 0.76≤ Light Sensor ≤1.00 ---> 3::
 
This table shows the final training data.

In [3]:
#group light sensor value into 4 level

for i in range (len(df)):
    if(df.loc[i, 'Light Sensor'] >= 0.0 and df.loc[i, 'Light Sensor'] <= 0.25):
        df.loc[i,'Light Sensor'] = 0
    if(df.loc[i, 'Light Sensor'] >= 0.26 and df.loc[i,'Light Sensor'] <= 0.50):
        df.loc[i,'Light Sensor'] = 1
    if(df.loc[i, 'Light Sensor'] >=0.51 and df.loc[i, 'Light Sensor'] <= 0.75):
        df.loc[i,'Light Sensor'] = 2
    if(df.loc[i, 'Light Sensor'] >= 0.76 and df.loc[i, 'Light Sensor'] <= 1.0):
        df.loc[i,'Light Sensor'] = 3
df['Light Sensor'] = df['Light Sensor'].astype('int64') #convert float64 into int64
df

,Light Sensor,Time,Light State
0,0,00:00,Off
1,0,01:00,Off
2,0,02:00,Off
3,0,03:00,Off
4,0,04:00,Off
5,0,04:00,Off
6,0,05:00,On
7,0,06:00,On
8,3,07:00,On
9,3,08:00,Off


Build the Model
Now, let's build our Naive Bayes model for predicting light state. Let  𝑋𝑖  where  𝑖={1,2}  be the features that correspond to light sensor and time, repectively. Then, let  𝑦  be the label (light state). So, you can make a prediction of light state using the following Naive Bayes equation:
𝑦̂ =argmax𝑦𝑃(𝑦)∏𝑖=1𝑛𝑃(𝑋𝑖|𝑦)
 
where  𝑦̂   is the prediction and  𝑛=2 .

Let's say light sensor level is 2 and time is 12:00, can you predict the light state? Well, let's apply the same steps as in the previous examples. Here, we want to calculate the following:

𝑦̂ =argmax𝑦(𝑃(𝑦=𝑜𝑓𝑓)𝑃(𝑋1=2|𝑦=𝑜𝑓𝑓)𝑃(𝑋2=12|𝑦=𝑜𝑓𝑓),𝑃(𝑦=𝑜𝑛)𝑃(𝑋1=2|𝑦=𝑜𝑛)𝑃(𝑋2=12|𝑦=𝑜𝑛))
 
This  argmax  fuction returns the predicted light state  𝑦  either "on" or "off". If the product of the first term is greater than the product of the second term, then  𝑦  is "off", otherwise if the product of the second term is greater than the product of the first term, then  𝑦  is "on".

Step 0: create a frequency table for each feature of the training data
First, we need to create a frequency table for each feature of the training data. Since we have two features (light sensor and time), we need to create two frequency tables.

From the training data, you need to count how many "Off" and "On" for each light sensor level. For example, in level 0 there are 6 "Off" and 12 "On". Then, you need to sum them up to get the total.

Light Sensor	Off	On	Total
Level 0	6	12	18
Level 1	4	5	9
Level 2	5	0	5
Level 3	5	0	5
Total	20	17	37
From the training data, you need to count how many "Off" and "On" for each hour from 00:00 to 23:00. Then, you need to sum them up to get the total.

Time	Off	On	Total
00:00	1	0	1
01:00	1	0	1
02:00	1	0	1
03:00	1	0	1
04:00	2	0	2
05:00	0	1	1
06:00	0	1	1
07:00	0	1	1
08:00	1	1	2
09:00	1	1	2
10:00	1	0	1
11:00	1	0	1
12:00	2	1	3
13:00	2	0	2
14:00	1	1	2
15:00	1	1	2
16:00	1	0	1
17:00	1	1	2
18:00	1	1	2
19:00	1	1	2
20:00	0	2	2
21:00	0	2	2
22:00	0	1	1
23:00	0	1	1
Total	20	17	37
Step 1: compute the probabilities for each value of the label
In this step, you need to compute the  𝑃(𝑦=𝑜𝑓𝑓)  and  𝑃(𝑦=𝑜𝑛) . Out of 37 observations, you have 20 'Off' and 17 'On'. So the respective probabilities are:
𝑃(𝑦=𝑜𝑓𝑓)=2037
 
𝑃(𝑦=𝑜𝑛)=1737
 
Step 2: compute the conditional probabilities
Out of 20 'Off', you have 5 'Level 2'. So the probability is:
𝑃(𝑋1=2|𝑦=𝑜𝑓𝑓)=520
 
Out of 20 'Off', you have 2 '12:00'. So the probability is:
𝑃(𝑋2=12|𝑦=𝑜𝑓𝑓)=220
 
Out of 17 'On', you have 0 'Level 2'. So the probability is:
𝑃(𝑋1=2|𝑦=𝑜𝑛)=017
 
Out of 17 'On', you have 1 '12:00'. So the probability is:
𝑃(𝑋2=12|𝑦=𝑜𝑛)=117
 
Because there are zero probabilities, you need to add Laplace Smoothing. For light sensor, you need to add 1 count to the numerator and add 4 times 1 to the denominator (4 is the number of possible levels in light sensor). For time, you need to add 1 count to the numerator and add 24 times 1 to the denominator (24 is the number of possible hours in a day).
𝑃(𝑋1=2|𝑦=𝑜𝑓𝑓)=5+120+4.1=624
 
𝑃(𝑋2=12|𝑦=𝑜𝑓𝑓)=2+120+24.1=344
 
𝑃(𝑋1=2|𝑦=𝑜𝑛)=0+117+4.1=121
 
𝑃(𝑋2=12|𝑦=𝑜𝑛)=1+117+24.1=241
 
Step 3: subtitute all the three probabilities into the Naive Bayes classifier
𝑦̂ =argmax𝑦(𝑃(𝑦=𝑜𝑓𝑓)𝑃(𝑋1=2|𝑦=𝑜𝑓𝑓)𝑃(𝑋2=12|𝑦=𝑜𝑓𝑓),𝑃(𝑦=𝑜𝑛)𝑃(𝑋1=2|𝑦=𝑜𝑛)𝑃(𝑋2=12|𝑦=𝑜𝑛)
 
𝑦̂ =argmax𝑦(2037.624.344,1737.121.241)
 
𝑦̂ =argmax𝑦(0.00921,0.00107)
 
𝑦̂ =𝑜𝑓𝑓
 
So, the predicted light state when light sensor level is 2 and time is 12:00 is "Off".